# Signal preprocessing

## Libraries

In [ ]:
from gwpy.timeseries import TimeSeries # time domain data array in gwpy
from gwpy.plot import Plot # plotting in gwpy
from scipy.signal import hann
from src.load.get_data import *

#### Q-Transform
The constant quality factor transform (CQT), introduced by J.C. Brown in 1988, is an interesting alternative to the windowed Fourier transform (STFT / Short Time Fourier Transform) or wavelets, for time-frequency analysis.

The constant-Q transform transforms a data series to the frequency domain. It is related to the Fourier transform. In general, the transform is well suited to musical data and proves useful where frequencies span several octaves.It is more useful in the identification of instruments.

Unlike the Fourier transform, but similar to the mel scale, the constant-Q transform (Wikipedia) uses a logarithmically spaced frequency axis. The original paper reference below

Judith C. Brown, "Calculation of a constant Q spectral transform," J. Acoust. Soc. Am., 89(1):425–434, 1991.

From Wikipedia - In mathematics and signal processing, the constant-Q transform, simply known as CQT transforms a data series to the frequency domain. It is related to the Fourier transform[1] and very closely related to the complex Morlet wavelet transform. In general, the transform is well suited to musical data, and this can be seen in some of its advantages compared to the fast Fourier transform. As the output of the transform is effectively amplitude/phase against log frequency, fewer frequency bins are required to cover a given range effectively, and this proves useful where frequencies span several octaves. As the range of human hearing covers approximately ten octaves from 20 Hz to around 20 kHz, this reduction in output data is significant.

A Constant Q transform is a variation on the Discrete Fourier Transform (DFT). In other words, it is a type of wavelet transform.

I only have a casual understanding of both types of transforms myself, so take what I'm saying with a grain of salt.

A standard DFT uses a constant window size throughout all frequencies. This typically leads to a pretty consistent, fully continuous transform. However, the constant bin size for all frequencies leads to some problems when you map frequency on a logarithmic scale. Specifically, peaks on the lower end are incredibly wide (sometimes up to half an octave), lacking any sort of detail.

This is an issue for emulating human perception because humans perceive frequency on a logarithmic scale.

A Constant Q transform seeks to solve this problem by increasing the window size for lower frequencies, and alleviate some of the computational strain caused by this by reducing the window size used for high frequencies. It's pretty effective at this, but has a few drawbacks.

The computational complexity of a Constant Q transform is only slightly larger than that of a standard DFT, but because the window size changes per frequency, it is impossible to apply the typical optimizations of the FFT to a Constant Q transform.

In other words, a Constant Q transform will yield better results where low frequencies and logarithmic frequency mapping are concerned.

The transform exhibits a reduction in frequency resolution with higher frequency bins, which is desirable for auditory applications. The transform mirrors the human auditory system, whereby at lower-frequencies spectral resolution is better, whereas temporal resolution improves at higher frequencies.

CQT refers to a time-frequency representation where the frequency bins are geometrically spaced and the Q-factors (ratios of the center frequencies to bandwidths) of all bins are equal.

## Setup variables

In [ ]:
train_data = pd.read_csv("data/data_path.csv")
train_data.head()

In [ ]:
# draw a random sample from the train data
sample_gw_id = train_data[train_data['target'] == 1].sample(random_state=42)['id'].values[0]

In [ ]:
# draw another random sample from train without gravitational wave signal
sample_no_gw_id = train_data[train_data['target'] == 0].sample(random_state=42)['id'].values[0]

In [ ]:
sample_rate = 2048

## Signal preprocessing

##### Typical signal processing workflow
Next, we try to implement the steps from this paper referenced above by following these steps:

- Plot the raw signal
- Window the signal
- Whiten the signal
- Bandpass the signal

In [ ]:
# Plot the raw signal
sample_gw_ts = TimeSeries(get_data(sample_gw_id)[0], sample_rate=sample_rate)
plot = sample_gw_ts.Plot()
ax = plot.gca()
ax.set_xlim(0, 2)

In [ ]:
# get a window of lenght of the signal
hann_win = hann(sample_rate*signal_length, False)
plt.plot(hann_win)

In [ ]:
# plot the windowed signal
sample_gw_ts_win = sample_gw_ts * hann_win
plot = sample_gw_ts_win.Plot()
ax = plot.gca()
ax.set_xlim(0, 2);

This is the windowed signal. Next, let's plot a whitened signal. As mentioned in the tutorial we referenced earlier, whitening the data is suppressing the extra noise at low frequencies and at the spectral lines, to better see the weak signals in the most sensitive band. Whitening is always one of the first steps in astrophysical data analysis (searches, parameter estimation). Whitening requires no prior knowledge of spectral lines, etc; only the data are needed.

In [ ]:
# plot the whitened signal
plot = sample_gw_ts.whiten().Plot()
ax = plot.gca()
ax.set_xlim(0, 2);

This is the whitened signal. Next, since we know this data is from merger binary black holes, the frequency is in lower range and this we apply a bandpass filter to passthrough signals between 35 ~ 350 Hz.

In [ ]:
# Bandpass the above whitened data and plot
plot = sample_gw_ts.whiten().bandpass(35, 350).Plot()
ax = plot.gca()
ax.set_xlim(0, 2);

### Signal Transformations - MFCC

In [ ]:
# plot the sample wit gravitational wave signal
visualize_sample_mfcc(train_data,sample_gw_id)

In [ ]:
# plot the sample without gravitational wave signal
visualize_sample_mfcc(train_data, sample_no_gw_id)

### Signal Transformations - Spectogram

In [ ]:
# plot the sample wit gravitational wave signal
visualize_sample_spectogram(train_data,sample_gw_id)

In [ ]:
# plot the sample without gravitational wave signal
visualize_sample_spectogram(train_data, sample_no_gw_id)

As seen in the image below, from the publication, our end signal looks very different to one the from the paper. 

To this end, let's see if we can get any info from the spectrogram images by tranforming the data using Constant Q-Transform.

### Signal Transformations - CQT

The signal analysis didn't provide much insights, so let's try the second method in signal processing. Tranforming the waves into spectrograms images, i.e. frequency-domain, and then visualize them. This technique is widely used in audio analysis (as shown here on this TensorFlow tutorial) and since our data is a wave with bunch of frequencies, we can use the same technique as well.

The advantage of using a spectrogram, over a direct Fourier Transform where you lose time info, is that it captures the shift or change in frequencies over time and this removes white noise frequencies that are persistent, leaving the signals of interest. Constant Q-Transform is one way to visualize the spectrogram.

In [ ]:
# let's plot two spectrograms for sample w/ and w/o GW signal side-by-side
plot_q_transform_sbs(sample_gw_id, sample_no_gw_id)

Visibly, all three signals have different features and the above were plotted from a sample which has gravitational waves, and it shows the famous 'chirp' confirming the presence of gravitational waves. This transformation removes the unwanted noise frequencies, but still some of it remains, but a signal has to be detected in all three waves to be predicted as gravitational wave.

Next, we can compare how the Q-Transforms look for samples with and without gravitational wave signals.

In [ ]:
# let's look at the sample with obvious "chirp"
# id from: https://www.kaggle.com/mistag/data-preprocessing-with-gwpy
plot_q_transform('0021f9dd71')

This clearly visible "chirp" can be seen with minimal background noise and is one of the good samples where the signal-to-ratio is quite strong, but as mentioned not all samples are like this and thus we use ML models to do this.

In [ ]:
fig2 = sample_gw_ts_win.asd(fftlength=2).plot(figsize=[12, 6])
plt.xlim(10,1024)
plt.ylim(1e-25, 1e-20);